In [54]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random
from sklearn.linear_model import LogisticRegression

In [56]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]

# Here we generate two arrays (currently lists), x_train and y_train.
# These will be the inputs into the machine learning model for training
# x_train is the array of data values that the model will use to predict a class
# x_train has 6 5-long lists that represent note values (explained in email)
# y_train is the array of correct classes and is filled with 0s and 1s
# if a vertical translation is present, the class is 1, and vice versa
# In the same fashion, we also generate x_test and y_test which will be used to test the model.

comp=[]
seq_len=3

# Make portion of simulated dataset without vertical translation
# There is a very small chance this will produce a few values with a translation, which are then classified correctly

for i in range(8000):
    xt=[]
    for j in range(2*seq_len):
        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(1, 8)])
    x_train.append(xt)
    for k in range(seq_len):
        comp.append(x_train[i][k+seq_len][4]-x_train[i][k][4])
    if comp.count(comp[0]) == len(comp):
        y_train.append(1)
    else:
        y_train.append(0)

# Produces section of dataset with vertical translations
        
for i in range(2000):
    xt=[]
    for j in range(seq_len):
        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(1, 8)])
    val=random.randint(1, 8)
    
    for j in range(seq_len):
        if xt[j][4]+val>8:
            xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), xt[j][3]+1, xt[j][4]+val-7])
        else:
            xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), xt[j][3], xt[j][4]+val])
    x_train.append(xt)
    y_train.append(1)
    
comp=[]


# Make portion of simulated dataset without vertical translation
# There is a very small chance this will produce a few values with a translation, which are then classified correctly

for i in range(8000):
    xt=[]
    for j in range(2*seq_len):
        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(1, 8)])
    x_test.append(xt)
    for k in range(seq_len):
        comp.append(x_test[i][k+seq_len][4]-x_test[i][k][4])
    if comp.count(comp[0]) == len(comp):
        y_test.append(1)
    else:
        y_test.append(0)

# Produces section of dataset with vertical translations
        
for i in range(2000):
    xt=[]
    for j in range(seq_len):
        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(1, 8)])
    val=random.randint(0, 9)
    
    for j in range(seq_len):
        if xt[j][4]+val>8:
            xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), xt[j][3]+1, xt[j][4]+val-8])
        else:
            xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), xt[j][3], xt[j][4]+val])
    x_test.append(xt)
    y_test.append(1)
print(x_test)
                

[[[0, 3, 1, 5, 1], [2, 0, 4, 4, 3], [0, 3, 4, 2, 5], [1, 7, 0, 2, 8], [0, 7, 3, 4, 8], [1, 6, 0, 1, 5]], [[0, 3, 0, 1, 3], [2, 4, 1, 4, 5], [2, 7, 0, 2, 6], [0, 0, 0, 4, 8], [2, 1, 3, 0, 6], [1, 0, 2, 2, 7]], [[1, 2, 3, 5, 3], [0, 4, 1, 1, 4], [1, 2, 1, 1, 1], [1, 4, 2, 5, 1], [1, 5, 0, 1, 4], [1, 2, 0, 1, 2]], [[1, 4, 1, 2, 8], [2, 0, 0, 5, 3], [0, 6, 2, 4, 5], [1, 3, 3, 1, 5], [1, 4, 4, 2, 5], [0, 3, 2, 3, 4]], [[0, 0, 0, 5, 8], [2, 5, 3, 4, 5], [2, 3, 3, 1, 3], [2, 3, 3, 3, 6], [0, 1, 3, 5, 4], [2, 3, 2, 2, 1]], [[1, 0, 4, 4, 7], [2, 0, 0, 4, 2], [1, 6, 3, 2, 2], [0, 7, 2, 3, 4], [2, 2, 2, 1, 7], [2, 3, 2, 0, 6]], [[1, 1, 4, 1, 2], [2, 6, 4, 4, 3], [1, 7, 3, 4, 3], [2, 7, 2, 1, 6], [0, 4, 1, 4, 4], [0, 7, 4, 0, 3]], [[1, 0, 4, 3, 5], [0, 5, 0, 4, 1], [0, 7, 2, 2, 1], [2, 4, 2, 4, 4], [2, 1, 3, 1, 7], [0, 6, 1, 5, 8]], [[1, 1, 0, 0, 8], [2, 1, 4, 0, 1], [2, 5, 3, 1, 1], [2, 7, 2, 4, 4], [2, 0, 3, 5, 5], [0, 2, 3, 0, 7]], [[2, 6, 1, 2, 5], [0, 5, 2, 5, 5], [2, 3, 0, 2, 6], [1, 4, 3, 4

In [3]:
'''
x_test=[]
y_test=[]
comp=[]
seq_len=3
for i in range(200000):
    xt=[]
    for j in range(2*seq_len):
        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(0, 9)])
    x_test.append(xt)
    for k in range(seq_len):
        comp.append(x_test[i][k+seq_len][4]-x_test[i][k][4])
    if comp.count(comp[0]) == len(comp):
        y_test.append(1)
    else:
        y_test.append(0)
'''

'\nx_test=[]\ny_test=[]\ncomp=[]\nseq_len=3\nfor i in range(200000):\n    xt=[]\n    for j in range(2*seq_len):\n        xt.append([random.randint(0, 2),random.randint(0, 7), random.randint(0,4), random.randint(0, 5), random.randint(0, 9)])\n    x_test.append(xt)\n    for k in range(seq_len):\n        comp.append(x_test[i][k+seq_len][4]-x_test[i][k][4])\n    if comp.count(comp[0]) == len(comp):\n        y_test.append(1)\n    else:\n        y_test.append(0)\n'

In [4]:
# convert to np arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [5]:
# reshape training data into understandable form
x_train=x_train.reshape(10000,30)
x_test=x_test.reshape(10000,30)

In [6]:
from sklearn.utils import shuffle
# shuffle both arrays in the same way to randomize
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_test, y_test = shuffle(x_test, y_test, random_state=0)

In [7]:
print(x_train.shape)

(10000, 30)


In [8]:
print(y_train.shape)

(10000,)


In [10]:
counter=0

# check number of vertical translations

for i in range(len(y_train)):
    if y_train[i]==1:
        counter+=1
print(counter)
    

2000


In [11]:
print(y_train)

[1 0 0 ... 0 1 0]


In [12]:
print(x_train[0])
# We can see that the note values are: 4, 3, 2, 3, 2, 1 (in this a vertical translation of 7 notes (the octave changes)). Thus, the class is 1.

[2 2 4 0 6 1 2 4 3 5 1 1 4 2 1 1 0 4 0 8 1 1 2 3 7 0 4 2 2 3]


In [13]:
model = LogisticRegression(random_state=0,max_iter=10000).fit(x_train,y_train)

In [14]:
model.score(x_test,y_test)

0.7945

In [25]:
from mido import MidiFile

mid = MidiFile('bach_846.mid', clip=True)
print(mid)

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='Das wohltemperierte Klavier I - Praeludium und Fuge 1 in C-Dur BWV 846', time=0),
    MetaMessage('copyright', text='Copyright © 1996  Bernd Krueger.', time=0),
    MetaMessage('text', text='Johann Sebastian Bach', time=0),
    MetaMessage('text', text='Fertiggestellt am 29.10.97\n', time=0),
    MetaMessage('text', text="Pan's auf Standard am 18.1.98\n", time=0),
    MetaMessage('text', text='Normierung: 23.12.2002\n', time=0),
    MetaMessage('text', text='Update am 25.9.2004\n', time=0),
    MetaMessage('text', text='Dauer: 3:46 Minuten\n', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=810811, time=0),
    MetaMessage('marker', text='Prelude', time=0),
    MetaMessage('set_tempo', tempo=794702, time=480),
    MetaMessag

In [38]:
print(len(mid.tracks))

11


In [39]:
for msg in mid.tracks[1]:
    print(msg)

MetaMessage('midi_port', port=0, time=0)
MetaMessage('track_name', name='Piano right', time=0)
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=127 time=0
MetaMessage('text', text='bdca426d104a26ac9dcb070447587523', time=0)
note_on channel=0 note=67 velocity=56 time=241
note_on channel=0 note=67 velocity=0 time=120
note_on channel=0 note=72 velocity=60 time=0
note_on channel=0 note=72 velocity=0 time=120
note_on channel=0 note=76 velocity=63 time=0
note_on channel=0 note=76 velocity=0 time=108
note_on channel=0 note=67 velocity=50 time=12
note_on channel=0 note=67 velocity=0 time=120
note_on channel=0 note=72 velocity=47 time=0
note_on channel=0 note=72 velocity=0 time=120
note_on channel=0 note=76 velocity=47 time=0
note_on channel=0 note=76 velocity=0 time=120
note_on channel=0 note=67 velocity=56 time=240
note_on channel=0 note=67 velocity=0 time

In [40]:
msgs=[]
for msg in mid.tracks[1]:
    if str(msg).startswith("note"):
        msgs.append(str(msg))

motif_msgs=[]
for msg1 in mid.tracks[1]:
    if str(msg1).startswith("note"):
        motif_msgs.append(str(msg1))



In [41]:
split_msgs=[]
motif_split_msgs=[]

for i in range(len(msgs)):
    x=msgs[i].split()
    if x[0]=="note_on":
        x[0]=0
    else:
        x[0]=1
    if x[3]=="velocity=0":
        x[0]=1
    x[2]=x[2].replace("note=","")
    x[4]=x[4].replace("time=","")
    split_msgs.append((x[0], int(x[2]),int(x[4])))
    print(split_msgs[i])
print("a")
for i in range(len(motif_msgs)):
    x=motif_msgs[i].split()
    if x[0]=="note_on":
        x[0]=0
    else:
        x[0]=1
    if x[3]=="velocity=0":
        x[0]=1
    x[2]=x[2].replace("note=","")
    x[4]=x[4].replace("time=","")
    motif_split_msgs.append((x[0], int(x[2]),int(x[4])))
    print(motif_split_msgs[i])

(0, 67, 241)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(0, 76, 0)
(1, 76, 108)
(0, 67, 12)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(0, 76, 0)
(1, 76, 120)
(0, 67, 240)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(0, 76, 0)
(1, 76, 108)
(0, 67, 12)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(0, 76, 0)
(1, 76, 1)
(0, 69, 240)
(1, 69, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 108)
(0, 69, 12)
(1, 69, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 120)
(0, 69, 240)
(1, 69, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 108)
(0, 69, 12)
(1, 69, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 1)
(0, 67, 240)
(1, 67, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 108)
(0, 67, 12)
(1, 67, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 120)
(0, 67, 240)
(1, 67, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 108)
(0, 67, 12)
(1, 67, 120)
(0, 74, 0)
(1, 74, 120)
(0, 77, 0)
(1, 77, 1)
(0, 67, 240)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(0, 76, 0)
(1, 76, 108)
(0, 67, 12)
(1, 67, 120)
(0, 72, 0)
(1, 72, 120)
(

In [58]:
import numpy as np
def dec_to_bin(x):
    return int(bin(x)[2:])

array_size=129

note_array=np.zeros([len(split_msgs), array_size], dtype=int)
motif_note_array=np.zeros([len(motif_split_msgs), array_size], dtype=int)
time=0
counter=0
time_array=np.zeros([len(split_msgs),21])

for note in split_msgs:
    note_array[counter][note[1]-1]=1 if note[0]==0 else 0
    x=dec_to_bin(time)
    print(x)
    count1=0
    for letter in str(x)[::-1]:
        time_array[counter][20-count1]=int(letter)
        count1+=1
    time+=note[2]
    counter = counter+1
    
counter=0
for note1 in motif_split_msgs:
    motif_note_array[counter][note1[1]-1]=1 if note1[0]==0 else 0
    counter = counter+1
print(motif_note_array[0:4])

0
11110001
101101001
101101001
111100001
111100001
1001001101
1001011001
1011010001
1011010001
1101001001
1101001001
1111000001
10010110001
10100101001
10100101001
10110100001
10110100001
11000001101
11000011001
11010010001
11010010001
11100001001
11100001001
11100001010
11111111010
100001110010
100001110010
100011101010
100011101010
100101010110
100101100010
100111011010
100111011010
101001010010
101001010010
101011001010
101110111010
110000110010
110000110010
110010101010
110010101010
110100010110
110100100010
110110011010
110110011010
111000010010
111000010010
111000010011
111100000011
111101111011
111101111011
111111110011
111111110011
1000001011111
1000001101011
1000011100011
1000011100011
1000101011011
1000101011011
1000111010011
1001011000011
1001100111011
1001100111011
1001110110011
1001110110011
1010000011111
1010000101011
1010010100011
1010010100011
1010100011011
1010100011011
1010100011100
1011000001100
1011010000100
1011010000100
1011011111100
1011011111100
1011101101000
10

In [43]:
import difflib
motif_s, motif_e= 0 , 23
motif_size=motif_e-motif_s

comparison_list=[]

motif=np.concatenate(motif_note_array[motif_s:motif_e])



for i in range(len(note_array)-motif_size):
    y=np.concatenate(note_array[i:i+motif_size])
    comparison_list.append(np.abs(np.subtract(motif, y)).sum())
    
    
print(comparison_list)
    

[0, 23, 24, 23, 24, 23, 6, 23, 24, 23, 24, 23, 12, 23, 24, 23, 24, 23, 18, 23, 24, 23, 24, 23, 24, 23, 24, 23, 24, 23, 22, 23, 24, 23, 24, 23, 20, 23, 24, 23, 24, 23, 18, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 12, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 4, 23, 24, 23, 24, 23, 0, 23, 24, 23, 22, 23, 6, 23, 24, 23, 20, 23, 12, 23, 24, 23, 18, 23, 18, 23, 24, 23, 16, 23, 24, 23, 24, 23, 18, 23, 24, 23, 24, 23, 20, 23, 24, 23, 24, 23, 22, 23, 24, 23, 24, 23, 24, 23, 24, 23, 24, 23, 24, 23, 22, 23, 24, 23, 24, 23, 20, 23, 24, 23, 24, 23, 18, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 18, 23, 20, 23, 24, 23, 20, 23, 16, 23, 24, 23, 22, 23, 12, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 8, 23, 24, 23, 24, 23, 10, 23, 24, 23, 24, 23, 12, 23, 24, 23, 24, 23, 14, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24, 23, 16, 23, 24, 23, 24

In [44]:
# for horizontal reflection
import numpy as np

array_size=129
split_msgs1=[[0,79,20],[1,79,20],[0,72,20],[1,72,20],[0,74,20], [1,74,20], [1,76,20],[0,76,20],[1,77,20],[0,77,20],[0,79,20],[1,79,20],[0,82,20],[1,82,20],[0,79,20],[1,79,20],[0,77,20],[1,77,20],[0,76,20],[1,76,20],[1,74,20],[0,74,20],[1,76,20],[0,76,20],[0,77,20],[1,77,20]]
note_array2=np.zeros([len(split_msgs1), array_size], dtype=int)
time=0
counter=0

for note in split_msgs1:
    note_array2[counter][note[1]-1]=1 if note[0]==0 else 0
    counter = counter+1



In [45]:
motif_s, motif_e= 0 , 23
motif_size=motif_e-motif_s

values=[]
for i in range(len(motif_split_msgs[motif_s:motif_e])):
    values.append(motif_split_msgs[i][1])
max_add=88-max(values)
max_sub=min(values)-22


comparison_list2=[]

for i in range(max_add):
    new_split=[]
    for j in range(motif_size):
        new_split.append([motif_split_msgs[j][0],motif_split_msgs[j][1]+i,motif_split_msgs[j][2]])
    counter=0
    
    motif=np.zeros([motif_size, array_size], dtype=int)
    
    for note in new_split:
        motif[counter][note[1]-1]=1 if note[0]==0 else 0
        counter=counter+1
        
    motif=np.concatenate(motif)
    x_list=[]
    
    for k in range(len(note_array)-motif_size):
        y=np.concatenate(note_array[k:k+motif_size])
        x_list.append(int(np.abs(np.subtract(motif,y)).sum()))
    comparison_list2.append(x_list)

    

for i in range(len(comparison_list2)):
    print(min(comparison_list2[i]))


0
16
8
16
16
16
16
8
23
8
16
23


In [50]:
motif_s, motif_e= 0 , 23
motif_size=motif_e-motif_s

values=[]
for i in range(len(motif_split_msgs[motif_s:motif_e])):
    values.append(split_msgs1[i][1])
max_add=88-max(values)
max_sub=min(values)-22


comparison_list2=[]

for i in range(max_sub):
    new_split=[]
    for j in range(motif_size):
        new_split.append([motif_split_msgs[j][0],motif_split_msgs[j][1]-i-1,motif_split_msgs[j][2]])
    counter=0
    
    motif=np.zeros([motif_size, array_size], dtype=int)
    
    for note in new_split:
        motif[counter][note[1]-1]=1 if note[0]==0 else 0
        counter=counter+1
        
    motif=np.concatenate(motif)
    x_list=[]
    
    for k in range(len(note_array)-motif_size):
        y=np.concatenate(note_array[k:k+motif_size])
        x_list.append(int(np.abs(np.subtract(motif,y)).sum()))
    comparison_list2.append(x_list)

    

for i in range(len(comparison_list2)):
    print(min(comparison_list2[i]))


16
16
8
16
0
16
8
16
16
8
16
0
16
16
16
16
8
16
16
23
16
22
22
23
23
22
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23


In [45]:
print(comparison_list2[0].index(0))

ValueError: 0 is not in list

In [60]:
for j in range(len(comparison_list2)):
    for i in range(len(comparison_list2[j])):
        if comparison_list2[j][i]<=10:
            print(j, i, comparison_list2[j][i])
    print('---')


---
---
2 264 8
2 270 10
---
---
4 228 8
4 234 4
4 240 0
4 246 4
4 252 8
4 306 10
4 312 8
4 318 10
---
---
6 362 8
6 368 8
6 374 8
6 380 8
6 386 8
6 392 10
6 596 10
6 602 8
6 608 10
6 692 10
6 698 8
6 704 10
---
---
---
9 214 8
9 220 10
9 648 8
---
---
11 420 8
11 426 4
11 432 0
11 438 2
11 444 4
11 450 6
11 456 8
11 462 8
11 468 8
11 474 8
11 480 8
11 486 10
11 564 8
11 570 4
11 576 0
11 582 2
11 588 4
11 594 6
11 600 8
11 606 10
11 666 10
11 672 8
11 678 8
11 684 8
11 690 8
11 696 8
11 702 10
11 738 10
11 744 8
---
---
---
---
---
16 406 8
16 412 10
16 550 8
16 556 10
16 616 10
16 622 8
16 712 10
16 718 8
16 724 10
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---


In [91]:
motif_s, motif_e= 0 , 25
motif_size=motif_e-motif_s

values=[]
for i in range(len(split_msgs)):
    values.append(split_msgs[i][1])
max_add=88-max(values)
max_sub=min(values)-22


comparison_list2=[]

x=np.concatenate(note_array2[motif_s:motif_e])

for i in range(max_sub):
    new_split=[]
    for j in range(len(split_msgs1)):
        new_split.append([split_msgs1[j][0],split_msgs1[j][1]-i,split_msgs1[j][2]])
    counter=0
    array_size=129
    note_array1=np.zeros([len(split_msgs1), array_size], dtype=int)
    
    for note in new_split:
        note_array1[counter][note[1]-1]=1 if note[0]==0 else 0
        counter = counter+1
    
    x_list=[]
    for k in range(len(note_array)-motif_size):
        y=np.concatenate(note_array1[k:k+motif_size])
        x_list.append(int(np.abs(np.subtract(x, y)).sum()))
    comparison_list2.append(x_list)

for i in range(len(comparison_list2)):
    print(min(comparison_list2[i]))


ValueError: operands could not be broadcast together with shapes (3225,) (3096,) 